In [ ]:
from prefect import flow, task
from prefect.task_runners import DaskTaskRunner
from typing import List
import httpx

@task(retries=3)
def get_stars(repo: str):
    url = f"https://api.github.com/repos/{repo}"
    count = httpx.get(url).json()["stargazers_count"]
    print(f"{repo} has {count} stars!")

@flow(name="Github Stars", task_runner=DaskTaskRunner())
def github_stars(repos: List[str]):
    for repo in repos:
        get_stars(repo)

@flow(name="Github Stats")
def github_stats():
    github_stars(["PrefectHQ/Prefect", "PrefectHQ/miter-design"])

# run the flow!
if __name__ == "__main__":
    github_stars()